In [ ]:
!pip install ortools
!pip install folium

In [ ]:
import pandas as pd
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium

In [ ]:
#Preparación de los datos

# Definir ciudades colombianas con sus coordenadas
data = {
    'Ciudad': ['Bogotá', 'Medellín', 'Cali', 'Barranquilla', 'Cartagena', 'Bucaramanga', 'Pereira', 'Manizales', 'Armenia'],
    'Latitud': [4.7110, 6.2442, 3.4372, 10.9685, 10.3910, 7.1193, 4.8087, 5.0703, 4.5339],
    'Longitud': [-74.0721, -75.5812, -76.5320, -74.7813, -75.4794, -73.1227, -75.6961, -75.5138, -75.6811]
}

ciudades = pd.DataFrame(data)
ciudades

,Ciudad,Latitud,Longitud
0,Bogotá,4.7110,-74.0721
1,Medellín,6.2442,-75.5812
2,Cali,3.4372,-76.5320
3,Barranquilla,10.9685,-74.7813
4,Cartagena,10.3910,-75.4794
5,Bucaramanga,7.1193,-73.1227
6,Pereira,4.8087,-75.6961
7,Manizales,5.0703,-75.5138
8,Armenia,4.5339,-75.6811


In [ ]:
#Calcular distancias

def haversine(lon1, lat1, lon2, lat2):
    # Convertir de grados a radianes
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Fórmula de Haversine
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

# Crear una matriz de distancias
num_ciudades = len(ciudades)
distancias = np.zeros((num_ciudades, num_ciudades))

for i in range(num_ciudades):
    for j in range(num_ciudades):
        distancias[i, j] = haversine(ciudades['Longitud'][i], ciudades['Latitud'][i], ciudades['Longitud'][j], ciudades['Latitud'][j])

distancias


array([[  0.        , 238.52311388, 307.21369929, 699.73000118,
        649.93480975, 287.45901087, 180.17214377, 164.54325696,
        179.30195671],
       [238.52311388,   0.        , 329.21269424, 532.28979405,
        460.9494876 , 288.24193159, 160.02553096, 130.66251621,
        190.37834689],
       [307.21369929, 329.21269424,   0.        , 858.85184452,
        781.40667648, 556.49971802, 178.35824551, 213.69483292,
        154.11020384],
       [699.73000118, 532.28979405, 858.85184452,   0.        ,
         99.64820648, 464.84619948, 691.86779742, 660.3710655 ,
        721.86951638],
       [649.93480975, 460.9494876 , 781.40667648,  99.64820648,
          0.        , 446.27186775, 620.79239137, 591.27551582,
        651.24986696],
       [287.45901087, 288.24193159, 556.49971802, 464.84619948,
        446.27186775,   0.        , 383.16152014, 348.77492281,
        403.14228409],
       [180.17214377, 160.02553096, 178.35824551, 691.86779742,
        620.79239137, 383.1615

In [ ]:
# Resolver el TSP

def create_data_model():
    data = {}
    data['distance_matrix'] = distancias
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

def print_solution(manager, routing, solution):
    index = routing.Start(0)
    plan_output = 'Ruta:\n'
    route_distance = 0
    route = []
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        route.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    route.append(manager.IndexToNode(index))
    plan_output += 'Distancia de la ruta: {} km\n'.format(route_distance)
    print(plan_output)
    return route

def solve_tsp():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        return manager, routing, solution
    else:
        return None, None, None

manager, routing, solution = solve_tsp()
if manager and routing and solution:
    route = print_solution(manager, routing, solution)
else:
    print("No se encontró solución para el TSP.")


Ruta:
 0 -> 8 -> 7 -> 6 -> 5 -> 4 -> 3 -> 2 -> 1 -> 0
Distancia de la ruta: 0 km



In [ ]:
def plot_route(ciudades, route):
    m = folium.Map(location=[4.5709, -74.2973], zoom_start=6)
    for i, row in ciudades.iterrows():
        folium.Marker([row['Latitud'], row['Longitud']], popup=row['Ciudad']).add_to(m)

    for i in range(len(route)-1):
        start_city = ciudades.iloc[route[i]]
        end_city = ciudades.iloc[route[i+1]]
        folium.PolyLine(locations=[(start_city['Latitud'], start_city['Longitud']), (end_city['Latitud'], end_city['Longitud'])], color='blue').add_to(m)

    return m

if 'route' in locals():
    m = plot_route(ciudades, route)
    m.save("ruta_optima.html")
    m
else:
    print("No se puede visualizar la ruta porque no se encontró solución.")